<a href="https://colab.research.google.com/github/ayaanzhaque/SuiSense/blob/master/deployedBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [45]:
!pip install transformers

In [25]:
#importing relevant libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
import torch

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

import transformers as ppb
import warnings
warnings.filterwarnings('ignore')




#cars = {0: [""]}

#happy
#cars = {0: ["It might seem crazy what I am 'bout to saySunshine, she's here, you can take a breakI'm a hot air balloon that could go to spaceWith the air, like I don't care, baby by the wayHuh (Because I'm happy)Clap along if you feel like a room without a roof(Because I'm happy)Clap along if you feel like happiness is the truth(Because I'm happy)Clap along if you know what happiness is to you(Because I'm happy)Clap along if you feel like that's what you wanna doHere come bad news talking this and that (Yeah)Well give me all you got, don't hold back (Yeah)Well I should probably warn you I'll be just fine (Yeah)No offense to you don't waste your timeHere's whyClap along if you feel like a room without a roof(Because I'm happy)Clap along if you feel like happiness is the truth(Because I'm happy)Clap along if you know what happiness is to you(Because I'm happy)Clap along if you feel like that's what you wanna doUh, bring me downCan't nothing, bring me downMy level's too high to bring me downCan't nothing, bring me down, I saidBring me down, can't nothingBring me downMy level's too high to bring me downCan't nothing, bring me down, I saidClap along if you feel like a room without a roof(Because I'm happy)Clap along if you feel like happiness is the truth(Because I'm happy)Clap along if you know what happiness is to you(Because I'm happy)Clap along if you feel like that's what you wanna doClap along if you feel like a room without a roof(Because I'm happy)Clap along if you feel like happiness is the truthClap along if you feel like that's what you wanna do"]}

#one more light -Linkin park

cars = {0: ["Should've stayed, were there signs, I ignored?Can I help you, not to hurt, anymore?We saw brilliance, when the world, was asleepThere are things that we can have, but can't keepIf they sayWho cares if one more light goes out?In a sky of a million starsIt flickers, flickersWho cares when someone's time runs out?If a moment is all we areWe're quicker, quickerWho cares if one more light goes out?Well I doThe reminders pull the floor from your feetIn the kitchen, one more chair than you need ohAnd you're angry, and you should be, it's not fairJust 'cause you can't see it, doesn't mean it, isn't thereIf they sayWho cares if one more light goes out?In a sky of a million starsIt flickers, flickersWho cares when someone's time runs out?If a moment is all we areWe're quicker, quickerWho cares if one more light goes out?Well I doWho cares if one more light goes out?In a sky of a million starsIt flickers, flickersWho cares when someone's time runs out?If a moment is all we areWe're quicker, quickerWho cares if one more light goes out?Well I doWell I do"],}
batch_1 = pd.DataFrame(cars, columns = [0])
print(batch_1)

# For DistilBERT:
#model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
## Want BERT instead of distilBERT? Uncomment the following line:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

tokenized = batch_1[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=128)))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

np.array(padded).shape

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

features = last_hidden_states[0][:,0,:].numpy()
print(features)



                                                   0
0  Should've stayed, were there signs, I ignored?...


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[[-2.91771293e-01  1.91371813e-01 -3.32322776e-01 -2.01606765e-01
  -2.53624976e-01  4.12177667e-02  3.74816358e-01  1.51264817e-01
  -1.17346101e-01 -5.89671433e-01  2.39425987e-01 -2.35560015e-01
  -1.03007264e-01  2.94457346e-01 -1.60163403e-01  3.19945097e-01
   1.50792882e-01  2.47040719e-01  4.60696295e-02  4.39773723e-02
   1.45368159e-01 -5.00449911e-02  2.98512191e-01 -9.57109928e-02
   1.68500617e-01 -2.19800938e-02  1.74197555e-01 -4.02473956e-01
   1.31069303e-01 -8.65847021e-02 -1.89728096e-01  4.81657237e-01
  -1.19277880e-01 -3.53675216e-01  4.81284298e-02 -8.74335617e-02
  -2.04273850e-01 -1.20782182e-02  1.02231711e-01  1.36434972e-01
  -2.46922970e-01  1.46238089e-01 -4.17684168e-01 -1.80500090e-01
   8.10365528e-02 -3.38752456e-02 -2.95957661e+00 -8.59258994e-02
  -2.10242316e-01 -7.74717927e-02  2.81858325e-01 -2.90021211e-01
   1.14152417e-01  1.67795330e-01  1.35354161e-01  3.78472716e-01
  -9.92384180e-02  4.62184995e-01  3.60569388e-01  1.78549841e-01
   1.20310

In [26]:
from keras.models import load_model

model = load_model('/Users/haridevaraj/Documents/Junction/SuiSense-master/notebooks/models/baseline_model.h5')
#realSuicidal = "According to our algorithm, the text has been classified as suicidal."
#realDepression = "According to our algorithm, the text has been classified as depression, not suicidal."



In [27]:
test = model.predict(features)

# 1- depression/suicidal , 0 - not
print(test)

1/1 [==============================] - 1s 678ms/step
[[0.5095358]]
